In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
# !pip install snowflake-connector-python
!pip uninstall cloudpickle -y
!pip install cloudpickle==2.2.1

In [1]:
from snowflake.snowpark import Session

In [2]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWINSIGHT"

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

In [5]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [6]:
!pip list | grep -i cloud

cloudpickle                2.2.1      
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
session.sql("LIST STAGES").show()

In [ ]:
# model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [14]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
df = registry.show_models()
df
# type(df)

# Get model list from snowpark registry

In [ ]:
model_list = registry.models()
model_list

# Get model details from registry

In [ ]:
m =registry.get_model("sample_sklearn_model")
m

# Get model versions

In [ ]:
df_version = m.show_versions()
df_version

# Set default model version

In [ ]:
# default_version = m.default
m.default = "V1"

# Get default model version details from model-registry

In [ ]:
mv = m.default
mv

In [ ]:
x

In [ ]:
#Load default vesion as python object
# clf = mv.load()

# Run prediction on scikit-learn model

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(x_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

# Deploy model with custom class

In [ ]:
from snowflake.ml.model import custom_model

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
#     def __init__(self, context: ModelContext) -> None:
#         super().__init__(context)
#         v = int(open(context.path('config')).read())
#         self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def run(self, data_list: list) -> pd.DataFrame:
#         features = self.context.model_ref('feature_preproc').transform(input)
#         model_output = self.context.model_ref('m2').predict(
#             self.context.model_ref('m1').predict(features)
#         )
#         return pd.DataFrame({
#             'output': model_output + self.bias})
        data_array = np.asarray(data_list)
        try:
            prediction = model.predict(data_array)
        except:
            prediction = model.predict(data_array.reshape(1, -1))
        return prediction.tolist()

In [ ]:
!pip freeze | grep cloud

# Train & register model

In [7]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [8]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

,short_name,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Argentina,94,94,565000,95500000,32,170,72,88,...,75,96,33,37,26,6,11,15,14,8
1,Cristiano Ronaldo,Portugal,93,93,405000,58500000,34,187,83,84,...,85,95,28,32,24,7,11,15,14,11
2,Neymar Jr,Brazil,92,92,290000,105500000,27,175,68,87,...,90,94,27,26,29,9,9,15,15,11
3,J. Oblak,Slovenia,91,93,125000,77500000,26,188,87,13,...,11,68,27,12,18,87,92,78,90,89
4,E. Hazard,Belgium,91,91,470000,90000000,28,175,74,81,...,88,91,34,27,22,11,12,6,8,8
5,K. De Bruyne,Belgium,91,91,370000,90000000,28,181,70,93,...,79,91,68,58,51,15,13,5,10,13
6,M. ter Stegen,Germany,90,93,250000,67500000,27,187,85,18,...,25,70,25,13,10,88,85,88,88,90
7,V. van Dijk,Netherlands,90,91,200000,78000000,27,193,92,53,...,62,89,91,92,85,13,10,13,11,11
8,L. Modric,Croatia,90,90,340000,45000000,33,172,66,86,...,82,92,68,76,71,13,9,7,14,9
9,M. Salah,Egypt,90,90,240000,80500000,27,175,71,79,...,77,91,38,43,41,14,14,9,11,14


In [9]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [12]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

GradientBoostingRegressor()

In [ ]:
type(gbm_default)

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_3")

In [15]:
mv = registry.log_model(gbm_default,
                   model_name="sample_sklearn_model_3",
                   version_name="v1",
                   conda_dependencies=["scikit-learn", "cloudpickle"],
                   comment="My awesome ML model",
                    python_version="3.9",
                   metrics={"score": 96},
                   sample_input_data=X_train
                       )

/opt/conda/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/tmp/pip_packages/snowflake/ml/model/model_signature.py:69: UserWarning: The sample input has 3750 rows, thus a truncation happened before inferring signature. This might cause inaccurate signature inference. If that happens, consider specifying signature manually.
  warnings.warn(


In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_3")

# Create stage in snowflake-model-registry

In [ ]:
# query = """
#         create or replace stage sample_model_serving_udf_stg
#         directory = (enable = true)
#         copy_options = (on_error='skip_file')
#         """

# session.sql(query)



In [ ]:
# To see the stages
session.sql("show stages").show()

In [ ]:
from snowflake.snowpark.files import SnowflakeFile

In [ ]:
import cloudpickle
with SnowflakeFile.open("@STG_ML_MODELS/ml_model_2") as file:
    m= cloudpickle.load(file)
    print(type(m))

# Upload model file on ML_MODELS stage 

In [ ]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE STG_ML_MODELS').show()

In [29]:
session.file.put(
    X_train.to_csv(), "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

SnowparkSQLException: (1304): 253006: File doesn't exist: [',age,height_cm,weight_kg,skill_dribbling,attacking_crossing,attacking_finishing,movement_acceleration,movement_sprint_speed,power_shot_power,mentality_aggression,defending_marking,defending_standing_tackle,goalkeeping_diving,goalkeeping_handling,overall,potential\n620,25,185,88,78,65,81,77,75,81,63,37,26,9,10,79,82\n2193,26,184,77,64,47,41,65,64,56,86,73,75,14,14,74,77\n1170,32,182,84,85,61,80,75,67,77,51,25,30,6,16,77,77\n1863,34,183,73,73,63,79,76,49,79,51,57,19,8,10,75,75\n4724,24,176,70,71,66,63,74,77,70,61,64,71,10,8,70,76\n2098,22,173,66,80,72,65,88,80,70,32,24,20,6,6,74,81\n769,25,178,70,84,77,70,78,72,79,56,24,44,13,12,78,83\n1200,28,186,76,74,49,80,64,66,75,58,22,29,11,11,77,77\n3539,35,178,83,71,73,61,83,86,67,72,66,72,12,12,72,72\n3035,29,180,77,76,68,63,77,84,86,78,50,53,16,7,73,73\n1349,28,185,73,13,30,11,58,58,56,21,20,12,74,73,76,78\n2801,27,179,72,77,71,59,76,66,68,50,42,52,15,12,73,74\n1464,30,194,92,12,11,13,

In [ ]:
session.file.put(
    "/notebooks/notebooks/ml_model_3", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

# Read model file from stage

In [ ]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@STG_ML_MODELS/ml_model_2")

#map packege dependancies
session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")

In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F
from typing import  Optional



def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
def read_model_from_file(file_name:Optional[str]) -> str:
    import numpy as np
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    data_array = np.asarray(data_list)
    pipeline = read_file(file_name)
    print("type of pickle object =", type(pipeline))
#     dict_mod["model_obj"] = pipeline
    return "pipeline"
#     return pipeline.predict(data_array).to_list()




# @F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
# def read_model_from_file(file_name:Optional[str]) -> str:
#     import numpy as np
    
#     # later we will input train data as JSON object
#     # hance, we have to convert JSON object as pandas DF
#     data_array = np.asarray(data_list)
#     pipeline = read_file(file_name)
#     print("type of pickle object =", type(pipeline))
# #     dict_mod["model_obj"] = pipeline
#     return pipeline
# #     return pipeline.predict(data_array).to_list()

In [ ]:
session.sql("DESC FUNCTION READ_MODEL_FROM_FILE("ml_model")").show()

In [ ]:
model_udf = read_model_from_file("ml_model_2")
# model_udf = read_file("ml_model_2")

In [ ]:
model_udf

In [ ]:
registry.

In [ ]:
import joblib

In [ ]:
# model_obj = joblib.load(read_model_from_file("ml_model_2"))
import os
model_file = os.path.join("/notebooks/notebooks", "ml_model_3")
model_obj = joblib.load(model_file)

In [ ]:
X_train.to_csv("/data/X_train.csv")

In [ ]:
import numpy as np

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = model_obj.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
from snowflake.snowpark import DataFrame

In [26]:
import pandas as pd

In [30]:
from snowflake.snowpark.types import Variant
def read_model_from_stage(session: Session, df_dict: dict) -> Variant:

    # print("file_name =", file_name)
    # session.add_import("@STG_ML_MODELS/"+file_name)
    # map packege dependancies
    # session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")



    # from snowflake.snowpark.types import PandasSeries, PandasDataFrame
    # import snowflake.snowpark.functions as F
    # from typing import Optional
    from snowflake.snowpark.files import SnowflakeFile
    from snowflake.ml.registry.registry import Registry

    import joblib
#     df = pd.DataFrame(df_dict)
    print("df ===", df)

    model_file = '@STG_ML_MODELS/ml_model_3'
    # Specify 'mode = rb' to open the file in binary mode.
    with SnowflakeFile.open(model_file, 'rb', require_scoped_url=False) as f:
        model_obj = joblib.load(f)
        # return str(model_obj)
        # return str(model_obj)

        reg = Registry(session=session)

        mv = reg.log_model(model=model_obj,
                           model_name="sample_model",
                           comment="test",
                           version_name="run1",
                           python_version="3.9.19",
                           conda_dependencies=["scikit-learn==1.3.2"],
                           metrics={"model_metrics": {"score": 96}, "project_id": "0001", "type": "Model"},
                           sample_input_data=df
                           )

        print("model got registered successfully")
        return df_dict

NameError: name 'DataFrame' is not defined

In [31]:
session.sproc.register(func=read_model_from_stage,
                           name="read_model_from_stage",
                           packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn", "pandas", "cloudpickle==2.2.1", "joblib"],
                           isPermanant=True,
                           stage_location="@STG_ML_MODELS",
                           replace=True,
                           # imports = ["@STG_ML_MODELS/ml_model_3"]
                       )

TypeError: Invalid return type or input types: return type VariantType(), input types [PandasDataFrameType()]

In [10]:
dict_df = X_train.to_dict()

In [11]:
X_train

,age,height_cm,weight_kg,skill_dribbling,attacking_crossing,attacking_finishing,movement_acceleration,movement_sprint_speed,power_shot_power,mentality_aggression,defending_marking,defending_standing_tackle,goalkeeping_diving,goalkeeping_handling,overall,potential
620,25,185,88,78,65,81,77,75,81,63,37,26,9,10,79,82
2193,26,184,77,64,47,41,65,64,56,86,73,75,14,14,74,77
1170,32,182,84,85,61,80,75,67,77,51,25,30,6,16,77,77
1863,34,183,73,73,63,79,76,49,79,51,57,19,8,10,75,75
4724,24,176,70,71,66,63,74,77,70,61,64,71,10,8,70,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,32,195,85,12,14,12,35,43,46,22,16,14,74,68,72,72
709,27,184,78,73,78,61,69,72,84,75,79,79,14,5,79,79
2487,29,180,71,73,64,72,55,52,70,82,47,64,10,15,74,74
174,23,171,69,84,84,50,86,84,79,68,75,78,7,13,83,87


In [18]:
df = pd.DataFrame(dict_df)

In [19]:
df

,age,height_cm,weight_kg,skill_dribbling,attacking_crossing,attacking_finishing,movement_acceleration,movement_sprint_speed,power_shot_power,mentality_aggression,defending_marking,defending_standing_tackle,goalkeeping_diving,goalkeeping_handling,overall,potential
620,25,185,88,78,65,81,77,75,81,63,37,26,9,10,79,82
2193,26,184,77,64,47,41,65,64,56,86,73,75,14,14,74,77
1170,32,182,84,85,61,80,75,67,77,51,25,30,6,16,77,77
1863,34,183,73,73,63,79,76,49,79,51,57,19,8,10,75,75
4724,24,176,70,71,66,63,74,77,70,61,64,71,10,8,70,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,32,195,85,12,14,12,35,43,46,22,16,14,74,68,72,72
709,27,184,78,73,78,61,69,72,84,75,79,79,14,5,79,79
2487,29,180,71,73,64,72,55,52,70,82,47,64,10,15,74,74
174,23,171,69,84,84,50,86,84,79,68,75,78,7,13,83,87


In [25]:
model_obj = session.call("read_model_from_stage", X_train)

SnowparkSQLException: (1304): 01b4b1d7-0000-65c9-0000-576d06c7d162: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 51, in compute
  File "<ipython-input-23-048f19e3b9a7>", line 30, in read_model_from_stage
  File "/usr/lib/python_udf/c9abc2f36199647a270d8ce824ea98bb92e1209b64d0ac5efb8c6c1d19a3e362/lib/python3.9/site-packages/snowflake/ml/_internal/telemetry.py", line 352, in wrap
    return update_stmt_params_if_snowpark_df(func(*args, **kwargs), statement_params)
  File "/usr/lib/python_udf/c9abc2f36199647a270d8ce824ea98bb92e1209b64d0ac5efb8c6c1d19a3e362/lib/python3.9/site-packages/snowflake/ml/registry/registry.py", line 156, in log_model
    return self._model_manager.log_model(
  File "/usr/lib/python_udf/c9abc2f36199647a270d8ce824ea98bb92e1209b64d0ac5efb8c6c1d19a3e362/lib/python3.9/site-packages/snowflake/ml/registry/_manager/model_manager.py", line 71, in log_model
    mc.save(
  File "/usr/lib/python_udf/c9abc2f36199647a2

# 